In [22]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [33]:
training_X = np.array([
    [1],
    [2],
    [3],
    [4],
    [5],
    [6],
    [7],
    [8],
    [9]
])

training_y = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])

validation_X = np.array([
    [10],
    [11],
    [12],
    [13],
    [14],
    [15],
    [16],
    [17],
    [18],
    [19]
])

validation_y = np.array([100, 110, 120, 130, 140, 150, 160, 170, 180, 190])

In [34]:
regressor = linear_model.SGDRegressor(max_iter = 1000, fit_intercept = True, eta0 = 0.01)

regressor.fit(training_X, training_y)

prediction_y = regressor.predict(validation_X)

print("Mean squared error: %.15f" % mean_squared_error(validation_y, prediction_y))
print(prediction_y)

Mean squared error: 0.022911147980220
[ 99.93641355 109.91883409 119.90125463 129.88367516 139.8660957
 149.84851623 159.83093677 169.8133573  179.79577784 189.77819837]


In [68]:
class SGDRegressor:
    
    def __init__(self, max_iteration = 1000, fit_intercept = False, learning_rate = 0.01):
        
        self.max_iteration = max_iteration
        self.fit_intercept = fit_intercept
        self.learning_rate = learning_rate
        
    def fit(self, X, y):
        
        if (self.fit_intercept):
            
            X = np.insert(X, 0, 1, axis = 1)
            
            self.thetas = np.zeros(X.shape[1])
        
        else:
            
            self.thetas = np.zeros(X.shape[1])
            
        for iteration in range(self.max_iteration):
            
            for inputs, targets in zip(X, y):
                
                print(inputs)
                
                print(targets)
                
                print(self.thetas)
                
                outputs = np.dot(self.thetas, inputs)
                
                self.thetas = self.thetas - (self.learning_rate * (outputs - targets) * inputs)
                
    def predict(self, X):
        
        outputs = np.array([])
        
        if (self.fit_intercept):
            
            X = np.insert(X, 0, 1, axis = 1)
        
        for inputs in X:
            
            output = np.dot(self.thetas, inputs)
            
            outputs = np.append(outputs, [output])
        
        return outputs
            
        

In [69]:
r = SGDRegressor(max_iteration = 1000, fit_intercept = True, learning_rate = 0.01)

r.fit(training_X, training_y)

prediction_y = r.predict(validation_X)

print(prediction_y)

print("Mean squared error: %.15f" % mean_squared_error(validation_y, prediction_y))

[1 1]
10
[0. 0.]
[1 2]
20
[0.1 0.1]
[1 3]
30
[0.297 0.494]
[1 4]
40
[0.57921 1.34063]
[1 5]
50
[0.9197927 2.7029608]
[1 6]
60
[1.27544673 4.48123096]
[1 7]
70
[1.59381841 6.39146101]
[1 8]
80
[1.83047795 8.04807783]
[1 9]
90
[1.96832695 9.15086978]
[1 1]
10
[2.0250654  9.66151583]
[1 2]
20
[2.00819958 9.64465002]
[1 3]
30
[1.99522459 9.61870003]
[1 4]
40
[1.98671134 9.59316029]
[1 5]
50
[1.98311782 9.57878619]
[1 6]
60
[1.98434733 9.58493375]
[1 7]
70
[1.98940783 9.61529676]
[1 8]
80
[1.99644298 9.6645428 ]
[1 9]
90
[2.00331513 9.71951997]
[1 1]
10
[2.00852518 9.76641043]
[1 2]
20
[1.99077582 9.74866108]
[1 3]
30
[1.97589484 9.71889912]
[1 4]
40
[1.96456892 9.68492135]
[1 5]
50
[1.95752638 9.65675118]
[1 6]
60
[1.95511355 9.64468707]
[1 7]
70
[1.95688119 9.65529291]
[1 8]
80
[1.96144188 9.6872177 ]
[1 9]
90
[1.96685004 9.73048302]
[1 1]
10
[1.97143807 9.77177527]
[1 2]
20
[1.95400594 9.75434314]
[1 3]
30
[1.93937902 9.72508929]
[1 4]
40
[1.92823255 9.69164989]
[1 5]
50
[1.92128423 9.66

10
[0.75773711 9.91228443]
[1 2]
20
[0.75103689 9.90558422]
[1 3]
30
[0.74541484 9.89434011]
[1 4]
40
[0.74113049 9.88148706]
[1 5]
50
[0.7384597  9.87080391]
[1 6]
60
[0.73753491 9.86617995]
[1 7]
70
[0.73818876 9.87010307]
[1 8]
80
[0.73989966 9.88207935]
[1 9]
90
[0.74193432 9.89835659]
[1 1]
10
[0.74366288 9.91391366]
[1 2]
20
[0.73708711 9.9073379 ]
[1 3]
30
[0.73156948 9.89630264]
[1 4]
40
[0.72736471 9.88368832]
[1 5]
50
[0.72474353 9.8732036 ]
[1 6]
60
[0.72383592 9.86866552]
[1 7]
70
[0.72447762 9.87251578]
[1 8]
80
[0.72615674 9.88426961]
[1 9]
90
[0.72815361 9.90024452]
[1 1]
10
[0.72985006 9.91551263]
[1 2]
20
[0.72339644 9.90905901]
[1 3]
30
[0.71798129 9.89822872]
[1 4]
40
[0.71385462 9.88584869]
[1 5]
50
[0.71128212 9.87555872]
[1 6]
60
[0.71039137 9.87110493]
[1 7]
70
[0.71102116 9.87488368]
[1 8]
80
[0.71266909 9.88641919]
[1 9]
90
[0.71462886 9.90209738]
[1 1]
10
[0.71629381 9.91708191]
[1 2]
20
[0.70996005 9.91074815]
[1 3]
30
[0.70464549 9.90011902]
[1 4]
40
[0.7005

[0.32351878 9.94307134]
[1 8]
80
[0.3242686  9.94832007]
[1 9]
90
[0.32516031 9.95545374]
[1 1]
10
[0.32591787 9.96227178]
[1 2]
20
[0.32303597 9.95938989]
[1 3]
30
[0.32061782 9.95455357]
[1 4]
40
[0.31877503 9.94902521]
[1 5]
50
[0.31762627 9.94443018]
[1 6]
60
[0.3172285  9.94244132]
[1 7]
70
[0.31750974 9.94412874]
[1 8]
80
[0.31824563 9.94927997]
[1 9]
90
[0.31912077 9.95628114]
[1 1]
10
[0.31986426 9.96297255]
[1 2]
20
[0.31703589 9.96014418]
[1 3]
30
[0.31466265 9.95539769]
[1 4]
40
[0.3128541  9.94997202]
[1 5]
50
[0.31172667 9.94546233]
[1 6]
60
[0.31133629 9.94351042]
[1 7]
70
[0.3116123  9.94516649]
[1 8]
80
[0.31233452 9.95022205]
[1 9]
90
[0.31319342 9.95709318]
[1 1]
10
[0.3139231 9.9636603]
[1 2]
20
[0.31114726 9.96088446]
[1 3]
30
[0.3088181  9.95622614]
[1 4]
40
[0.30704313 9.95090124]
[1 5]
50
[0.30593665 9.94647532]
[1 6]
60
[0.30555352 9.94455966]
[1 7]
70
[0.30582441 9.94618497]
[1 8]
80
[0.30653321 9.95114663]
[1 9]
90
[0.30737615 9.95789013]
[1 1]
10
[0.30809228 

60
[0.13142467 9.976154  ]
[1 7]
70
[0.13154119 9.97685308]
[1 8]
80
[0.13184606 9.97898719]
[1 9]
90
[0.13220862 9.9818877 ]
[1 1]
10
[0.13251665 9.98465989]
[1 2]
20
[0.13134488 9.98348812]
[1 3]
30
[0.13036167 9.9815217 ]
[1 4]
40
[0.1296124 9.9792739]
[1 5]
50
[0.12914532 9.97740558]
[1 6]
60
[0.12898359 9.97659692]
[1 7]
70
[0.12909794 9.97728301]
[1 8]
80
[0.12939715 9.97937748]
[1 9]
90
[0.12975298 9.98222412]
[1 1]
10
[0.13005528 9.98494481]
[1 2]
20
[0.12890528 9.98379481]
[1 3]
30
[0.12794033 9.98186492]
[1 4]
40
[0.12720498 9.97965886]
[1 5]
50
[0.12674657 9.97782525]
[1 6]
60
[0.12658784 9.97703161]
[1 7]
70
[0.12670007 9.97770496]
[1 8]
80
[0.12699372 9.97976052]
[1 9]
90
[0.12734294 9.98255429]
[1 1]
10
[0.12763963 9.98522445]
[1 2]
20
[0.12651099 9.98409581]
[1 3]
30
[0.12556396 9.98220176]
[1 4]
40
[0.12484227 9.98003668]
[1 5]
50
[0.12439238 9.97823712]
[1 6]
60
[0.1242366  9.97745822]
[1 7]
70
[0.12434674 9.97811907]
[1 8]
80
[0.12463494 9.98013645]
[1 9]
90
[0.124977

[1 9]
90
[0.04987148 9.99316771]
[1 1]
10
[0.04998767 9.99421343]
[1 2]
20
[0.04954566 9.99377142]
[1 3]
30
[0.04917477 9.99302965]
[1 4]
40
[0.04889214 9.99218174]
[1 5]
50
[0.04871594 9.99147698]
[1 6]
60
[0.04865494 9.99117193]
[1 7]
70
[0.04869807 9.99143074]
[1 8]
80
[0.04881094 9.99222081]
[1 9]
90
[0.04894516 9.99329462]
[1 1]
10
[0.0490592  9.99432091]
[1 2]
20
[0.04862539 9.99388711]
[1 3]
30
[0.0482614  9.99315912]
[1 4]
40
[0.04798401 9.99232696]
[1 5]
50
[0.04781109 9.99163528]
[1 6]
60
[0.04775122 9.99133591]
[1 7]
70
[0.04779355 9.99158991]
[1 8]
80
[0.04790432 9.9923653 ]
[1 9]
90
[0.04803605 9.99341916]
[1 1]
10
[0.04814797 9.9944264 ]
[1 2]
20
[0.04772223 9.99400065]
[1 3]
30
[0.04736499 9.99328618]
[1 4]
40
[0.04709275 9.99246948]
[1 5]
50
[0.04692305 9.99179065]
[1 6]
60
[0.04686429 9.99149683]
[1 7]
70
[0.04690583 9.99174612]
[1 8]
80
[0.04701455 9.99250711]
[1 9]
90
[0.04714383 9.9935414 ]
[1 1]
10
[0.04725367 9.99452992]
[1 2]
20
[0.04683583 9.99411208]
[1 3]
30
[

...

[1 1]
10
[1.00153823e-06 9.99999988e+00]
[1 2]
20
[9.92682224e-07 9.99999988e+00]
[1 3]
30
[9.85251281e-07 9.99999986e+00]
[1 4]
40
[9.79588443e-07 9.99999984e+00]
[1 5]
50
[9.76058333e-07 9.99999983e+00]
[1 6]
60
[9.74835990e-07 9.99999982e+00]
[1 7]
70
[9.75700221e-07 9.99999983e+00]
[1 8]
80
[9.77961598e-07 9.99999984e+00]
[1 9]
90
[9.80650901e-07 9.99999987e+00]
[1 1]
10
[9.82935628e-07 9.99999989e+00]
[1 2]
20
[9.74244117e-07 9.99999988e+00]
[1 3]
30
[9.66951196e-07 9.99999986e+00]
[1 4]
40
[9.61393540e-07 9.99999985e+00]
[1 5]
50
[9.57928999e-07 9.99999983e+00]
[1 6]
60
[9.56729359e-07 9.99999983e+00]
[1 7]
70
[9.57577538e-07 9.99999983e+00]
[1 8]
80
[9.59796912e-07 9.99999985e+00]
[1 9]
90
[9.62436264e-07 9.99999987e+00]
[1 1]
10
[9.64678554e-07 9.99999989e+00]
[1 2]
20
[9.56148479e-07 9.99999988e+00]
[1 3]
30
[9.48991018e-07 9.99999987e+00]
[1 4]
40
[9.43536590e-07 9.99999985e+00]
[1 5]
50
[9.40136399e-07 9.99999984e+00]
[1 6]
60
[9.38959041e-07 9.99999983e+00]
[1 7]
70
[9.3979

[3.48049376e-07 9.99999994e+00]
[1 6]
60
[3.47613505e-07 9.99999994e+00]
[1 7]
70
[3.47921678e-07 9.99999994e+00]
[1 8]
80
[3.48728055e-07 9.99999994e+00]
[1 9]
90
[3.49687025e-07 9.99999995e+00]
[1 1]
10
[3.50501728e-07 9.99999996e+00]
[1 2]
20
[3.47402451e-07 9.99999996e+00]
[1 3]
30
[3.44801893e-07 9.99999995e+00]
[1 4]
40
[3.42820107e-07 9.99999995e+00]
[1 5]
50
[3.41584698e-07 9.99999994e+00]
[1 6]
60
[3.41156923e-07 9.99999994e+00]
[1 7]
70
[3.41459372e-07 9.99999994e+00]
[1 8]
80
[3.42250771e-07 9.99999995e+00]
[1 9]
90
[3.43191929e-07 9.99999995e+00]
[1 1]
10
[3.43991500e-07 9.99999996e+00]
[1 2]
20
[3.40949789e-07 9.99999996e+00]
[1 3]
30
[3.38397534e-07 9.99999995e+00]
[1 4]
40
[3.36452557e-07 9.99999995e+00]
[1 5]
50
[3.35240095e-07 9.99999994e+00]
[1 6]
60
[3.34820265e-07 9.99999994e+00]
[1 7]
70
[3.35117096e-07 9.99999994e+00]
[1 8]
80
[3.35893796e-07 9.99999995e+00]
[1 9]
90
[3.36817473e-07 9.99999995e+00]
[1 1]
10
[3.37602192e-07 9.99999996e+00]
[1 2]
20
[3.34616978e-07 

[1.24984240e-07 9.99999999e+00]
[1 2]
20
[1.23879079e-07 9.99999998e+00]
[1 3]
30
[1.22951755e-07 9.99999998e+00]
[1 4]
40
[1.22245077e-07 9.99999998e+00]
[1 5]
50
[1.21804546e-07 9.99999998e+00]
[1 6]
60
[1.21652007e-07 9.99999998e+00]
[1 7]
70
[1.21759856e-07 9.99999998e+00]
[1 8]
80
[1.22042059e-07 9.99999998e+00]
[1 9]
90
[1.22377663e-07 9.99999998e+00]
[1 1]
10
[1.22662779e-07 9.99999999e+00]
[1 2]
20
[1.21578146e-07 9.99999998e+00]
[1 3]
30
[1.20668046e-07 9.99999998e+00]
[1 4]
40
[1.19974493e-07 9.99999998e+00]
[1 5]
50
[1.19542145e-07 9.99999998e+00]
[1 6]
60
[1.19392439e-07 9.99999998e+00]
[1 7]
70
[1.19498285e-07 9.99999998e+00]
[1 8]
80
[1.19775246e-07 9.99999998e+00]
[1 9]
90
[1.20104616e-07 9.99999998e+00]
[1 1]
10
[1.20384437e-07 9.99999999e+00]
[1 2]
20
[1.19319949e-07 9.99999998e+00]
[1 3]
30
[1.18426753e-07 9.99999998e+00]
[1 4]
40
[1.17746083e-07 9.99999998e+00]
[1 5]
50
[1.17321765e-07 9.99999998e+00]
[1 6]
60
[1.17174840e-07 9.99999998e+00]
[1 7]
70
[1.17278720e-07 

90
[5.16593936e-08 9.99999999e+00]
[1 1]
10
[5.17797500e-08 9.99999999e+00]
[1 2]
20
[5.13218927e-08 9.99999999e+00]
[1 3]
30
[5.09377112e-08 9.99999999e+00]
[1 4]
40
[5.06449413e-08 9.99999999e+00]
[1 5]
50
[5.04624338e-08 9.99999999e+00]
[1 6]
60
[5.03992383e-08 9.99999999e+00]
[1 7]
70
[5.04439192e-08 9.99999999e+00]
[1 8]
80
[5.05608329e-08 9.99999999e+00]
[1 9]
90
[5.06998705e-08 9.99999999e+00]
[1 1]
10
[5.08179912e-08 9.99999999e+00]
[1 2]
20
[5.03686382e-08 9.99999999e+00]
[1 3]
30
[4.99915925e-08 9.99999999e+00]
[1 4]
40
[4.97042605e-08 9.99999999e+00]
[1 5]
50
[4.95251429e-08 9.99999999e+00]
[1 6]
60
[4.94631213e-08 9.99999999e+00]
[1 7]
70
[4.95069724e-08 9.99999999e+00]
[1 8]
80
[4.96217145e-08 9.99999999e+00]
[1 9]
90
[4.97581694e-08 9.99999999e+00]
[1 1]
10
[4.98740963e-08 9.99999999e+00]
[1 2]
20
[4.94330896e-08 9.99999999e+00]
[1 3]
30
[4.90630472e-08 9.99999999e+00]
[1 4]
40
[4.87810521e-08 9.99999999e+00]
[1 5]
50
[4.86052615e-08 9.99999999e+00]
[1 6]
60
[4.85443920e-

[2.16777191e-08 1.00000000e+01]
[1 7]
70
[2.16969373e-08 1.00000000e+01]
[1 8]
80
[2.17472242e-08 1.00000000e+01]
[1 9]
90
[2.18070271e-08 1.00000000e+01]
[1 1]
10
[2.18578333e-08 1.00000000e+01]
[1 2]
20
[2.16645576e-08 1.00000000e+01]
[1 3]
30
[2.15023827e-08 1.00000000e+01]
[1 4]
40
[2.13787955e-08 1.00000000e+01]
[1 5]
50
[2.13017535e-08 1.00000000e+01]
[1 6]
60
[2.12750768e-08 1.00000000e+01]
[1 7]
70
[2.12939379e-08 1.00000000e+01]
[1 8]
80
[2.13432908e-08 1.00000000e+01]
[1 9]
90
[2.14019829e-08 1.00000000e+01]
[1 1]
10
[2.14518454e-08 1.00000000e+01]
[1 2]
20
[2.12621596e-08 1.00000000e+01]
[1 3]
30
[2.1102997e-08 1.0000000e+01]
[1 4]
40
[2.09817052e-08 1.00000000e+01]
[1 5]
50
[2.09060943e-08 1.00000000e+01]
[1 6]
60
[2.08799131e-08 1.00000000e+01]
[1 7]
70
[2.08984239e-08 1.00000000e+01]
[1 8]
80
[2.09468602e-08 1.00000000e+01]
[1 9]
90
[2.10044622e-08 1.00000000e+01]
[1 1]
10
[2.10533986e-08 1.00000000e+01]
[1 2]
20
[2.0867236e-08 1.0000000e+01]
[1 3]
30
[2.07110297e-08 1.00